# Estadistica aplicada 
## Lizbeth Ramos Saucedo 
### Sesion 10ma a - 07 marzo 2023 

In [31]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline 

In [32]:
from lifelines import CoxPHFitter

In [33]:
from lifelines.datasets import load_rossi

In [34]:
rossi= load_rossi()


In [35]:
rossi

,week,arrest,fin,age,race,wexp,mar,paro,prio
0,20,1,0,27,1,0,0,1,3
1,17,1,0,18,1,0,0,1,8
2,25,1,0,19,0,1,0,1,13
3,52,0,1,23,1,1,1,1,1
4,52,0,0,19,0,1,0,1,3
...,...,...,...,...,...,...,...,...,...
427,52,0,1,31,0,1,0,1,3
428,52,0,0,20,1,0,0,1,1
429,52,0,1,20,1,1,1,1,1
430,52,0,0,29,1,1,0,1,3


**week:** tiempo de supervivencia en semanas desde que se libero hasta que vuelve a ser arrestado 

**arrest:**(evento) 1 ai fue arrestado, 0 si no

**fin:** 1 si recibio ayuda en dinero, 0 no recibio 

**age:** edad cuando fue liberado 

**race:** 1 negro, 0 otra raza (white, hispanic, estc)

**wesp:** 1 cuando fue arrestado tenia trabajo, 0 no tenia trabajo 

**mar:** 1 si estaba casado al momento de la liberacion, 0 si no.

**paro:** libertad condicional 1, 0 

**prio:** encarcelamientos anteriores 


**Hipótesis**

1. Recibir ayuda ( dinero) dismunuye el riesgo de volver a ser arrestado.Si la persona tiene dinero no necesita cometer  ningún delito y por lo tanto no lo vuelven a arrestar.

2. Los jovenes son mas inexpertos y por tanto tienen mayor riesgo de volver a ser arrestados.

3. Ser negro incrementa el riesgo de volver a ser arrestado, por el racismo que hay en EE.UU.

4. Tener trabajo cuando te arrestaron disminuye el riesgo de volver a ser arrestado. Si tenias trabajo tienes experiencia laboral y sería más fácil que te contraten.

5. Estar casado, no quiere dejar abandonada a su pareja y no se arriesga a cometer delitos.

6. Si esta en libertad condicional, no quiere regresar a la cárcel por lo tanto no comete delitos y el risgo de ser arrestado disminuye.

7. Entre más veces haya estado en la cárcel es mayor el riesgo de ser arrestado.

In [23]:
rossi.describe()

,week,arrest,fin,age,race,wexp,mar,paro,prio
count,432.000000,432.000000,432.00000,432.000000,432.000000,432.000000,432.000000,432.000000,432.000000
mean,45.854167,0.263889,0.50000,24.597222,0.877315,0.571759,0.122685,0.618056,2.983796
std,12.662293,0.441251,0.50058,6.113375,0.328456,0.495398,0.328456,0.486426,2.896068
min,1.000000,0.000000,0.00000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50.000000,0.000000,0.00000,20.000000,1.000000,0.000000,0.000000,0.000000,1.000000
50%,52.000000,0.000000,0.50000,23.000000,1.000000,1.000000,0.000000,1.000000,2.000000
75%,52.000000,1.000000,1.00000,27.000000,1.000000,1.000000,0.000000,1.000000,4.000000
max,52.000000,1.000000,1.00000,44.000000,1.000000,1.000000,1.000000,1.000000,18.000000


In [26]:
arrestados = rossi.query('arrest==1') # query es un filtro 
no_arrest = rossi.query('arrest==0')

In [29]:
arrestados.describe()

,week,arrest,fin,age,race,wexp,mar,paro,prio
count,114.000000,114.0,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000
mean,28.710526,1.0,0.421053,22.763158,0.894737,0.456140,0.070175,0.596491,3.771930
std,14.447784,0.0,0.495908,5.123782,0.308247,0.500272,0.256570,0.492767,3.587182
min,1.000000,1.0,0.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,17.000000,1.0,0.000000,19.000000,1.000000,0.000000,0.000000,0.000000,1.000000
50%,28.000000,1.0,0.000000,21.000000,1.000000,0.000000,0.000000,1.000000,3.000000
75%,41.500000,1.0,1.000000,24.750000,1.000000,1.000000,0.000000,1.000000,5.000000
max,52.000000,1.0,1.000000,44.000000,1.000000,1.000000,1.000000,1.000000,18.000000


In [28]:
no_arrest.describe()

,week,arrest,fin,age,race,wexp,mar,paro,prio
count,318.0,318.0,318.000000,318.000000,318.000000,318.000000,318.000000,318.000000,318.000000
mean,52.0,0.0,0.528302,25.254717,0.871069,0.613208,0.141509,0.625786,2.701258
std,0.0,0.0,0.499985,6.309666,0.335652,0.487783,0.349096,0.484682,2.552057
min,52.0,0.0,0.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,52.0,0.0,0.000000,21.000000,1.000000,0.000000,0.000000,0.000000,1.000000
50%,52.0,0.0,1.000000,23.000000,1.000000,1.000000,0.000000,1.000000,2.000000
75%,52.0,0.0,1.000000,28.000000,1.000000,1.000000,0.000000,1.000000,3.000000
max,52.0,0.0,1.000000,44.000000,1.000000,1.000000,1.000000,1.000000,15.000000
